In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import requests
from sklearn.utils import resample
import math
import re
import time
import random
import collections
from collections import Counter
from sklearn.model_selection import train_test_split
import kshingle as ks
from sklearn.utils import shuffle

## Individual Assignment Computer Science COPY

Joost Alma
577925ja


# Goal: perform duplicate detection: Which product descriptions refer to the very same product.

Propose a scalable solution for the involved computations by reducing the number of comparisons using LHS
LHS: evaluate for efficacy (werkzaamheid) and efficiency

ModelID only for training

Use bootstrapping (with replacement) 63% training data, 37% test data
Take at least 5 bootstraps (robustness)


# Performance evaluation:
1. Average (across bootstrapps) F1-measure (harmonic mean between precision and recall)


Interpretation:
- TP: The pairs of products that are predicted to be duplicates and are real duplicates. 
- FP: is given by the pairs of products that are predicted to be duplicates but are real non-duplicates. 
- TN: is given by pairs of products that are predicted to be nonduplicates and are real non-duplicates. 
- FN: The pairs of products that are predicted to be non-duplicates but are real duplicates

# Importing dataset

In [2]:
with open("TVs-all-merged.json") as jsonFile:
    data = json.load(jsonFile)
    
#     jsonFile.close() #waarom nodig?
#     data = json.dumps(data, indent=2, sort_keys=True) #make data more readable

# Create list to be filled with datafile

In [3]:
file = []

##Add datafile to the list 'file'
for i in range(len(list(data.values()))):
    if len(list(data.values())[i]) ==1:
        file.append(list(data.values())[i][0])
    else:
        for j in range(len(list(data.values())[i])):
            file.append(list(data.values())[i][1:][0])
    
df = pd.DataFrame(file)
display(df)


,shop,url,modelID,featuresMap,title
0,bestbuy.com,http://www.bestbuy.com/site/Philips+-+4000+Ser...,29PFL4508/F7,"{'Screen Size Class': '29""', 'DVI Inputs': '0'...","Philips 4000 Series 29"" Class 2812"" Diag. LED ..."
1,newegg.com,http://www.newegg.com/Product/Product.aspx?Ite...,SC-3211,"{'Headphone': 'Yes', 'Remote Control': 'Full F...","Newegg.com - SuperSonic 32"" 720p LED HDTV SC-3211"
2,bestbuy.com,http://www.bestbuy.com/site/Sharp+-+AQUOS+-+90...,LC-90LE657U,"{'Language Options': 'English, French, Spanish...","Sharp AQUOS 90"" Class 90"" Diag. LED 1080p 120H..."
3,bestbuy.com,http://www.bestbuy.com/site/Philips+-+2000+Ser...,39PFL2908/F7,{'Meta Title': 'Philips Electronics 39PFL2908'...,"Philips 2000 Series 39"" Class 3858"" Diag. LED ..."
4,newegg.com,http://www.newegg.com/Product/Product.aspx?Ite...,LC70LE550U,"{'Remote Control': 'Yes', 'Series': 'Aquos', '...","Newegg.com - Sharp Aquos 70"" Class (69.5"" Diag..."
...,...,...,...,...,...
1619,bestbuy.com,http://www.bestbuy.com/site/Panasonic+-+VIERA+...,TC-P55GT31,"{'Internet Connectable': 'Yes', 'Language Opti...","Panasonic - VIERA 55"" Class / Plasma / 1080p ..."
1620,newegg.com,http://www.newegg.com/Product/Product.aspx?Ite...,UN40EH5000FXZA,"{'Component Video': '1 In', 'Energy Star Compl...","Samsung 40"" Class (40.0"" Diag.) 1080p 60Hz LED..."
1621,newegg.com,http://www.newegg.com/Product/Product.aspx?Ite...,UN40EH5000FXZA,"{'Component Video': '1 In', 'Energy Star Compl...","Samsung 40"" Class (40.0"" Diag.) 1080p 60Hz LED..."
1622,bestbuy.com,http://www.bestbuy.com/site/Insignia%26%23153%...,NS-32E859A11,"{'Internet Connectable': 'Yes', 'Language Opti...","Insignia - Connected TV 32"" Class / 1080p / 1..."


In [4]:
##extract all double modelID's from datafile
dupID = df.pivot_table(columns=['modelID'], aggfunc='size')
dupID = dupID[dupID > 1]
print("Total ammount of duplicates:", len(dupID))

##Enter a productID to check whether and how many times it is in the dataset 
def duplicateControl(A):
    if A in dupID:
        print(A, 'is contained in dupID', dupID[A], 'times')
    else:
        print(A, 'is not more than once in the dataset')
        
duplicateControl('22LN4510')


Total ammount of duplicates: 329
22LN4510 is contained in dupID 2 times


# Extract all words of the dataset and put in list to eliminate duplicates


# Bootstrapping, train & test datasets

In [5]:
##seperate the x_data and y_data

x_data = df[['shop', 'url', 'featuresMap', 'title']]
y_data = df['modelID']

In [6]:
##split x_data and y_data into x&y train & test data
##Traindataset is roughly 63% of the original dataset
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.37)
print('length xtrain', len(x_train)); print('length ytrain', len(y_train))
print('length xtest', len(x_test)); print('length ytest', len(y_test))

length xtrain 1023
length ytrain 1023
length xtest 601
length ytest 601


# Keywords & convert titles to words

Use title only for now

In [7]:
##First, make a function which converts the titles into a strings 
#https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/

##Function: input is the index number of the title to be converted into strings


def textfunc(A, index_sample):
    words = A.iloc[index_sample]
#     words = re.sub(r'[^\w\s]','', words)
    
    units = words.split()
    units = words.lower() #needed for modelID to be filtered
    return str(units)

print(textfunc(x_data['title'], 0)) #gives the title of the first product in x-trainset as strings
print(textfunc(y_data, 0))


philips 4000 series 29" class 2812" diag. led 720p 60hz hdtv 29pfl4508f7 - best buy
29pfl4508/f7


In [8]:
##create a list consisting of all modelID's to later filter them out of the titles for keywords
listIDs = []
for i in range(len(df)):
    listIDs.append(textfunc(y_data, i))  
    listIDs[i] = listIDs[i].replace('/', '')
    listIDs[i] = listIDs[i].replace('-', '')
listIDs[0:10]

['29pfl4508f7',
 'sc3211',
 'lc90le657u',
 '39pfl2908f7',
 'lc70le550u',
 'un40f6350a',
 'hg40na577lf',
 'un46f5000af',
 'e463',
 'un46es6580']

# Combine all titles
Create shingles from the titles

In [9]:
##create list to add all titles in string format
titles_string = []
for i in range(len(x_data)):
    titles_string.append(textfunc(x_data['title'], i))
    titles_string[i] = titles_string[i].replace('"', 'inch') #transform to inch
    titles_string[i] = titles_string[i].replace('inch diag', 'inch')
    titles_string[i] = titles_string[i].replace('inch.', 'inch') 
    titles_string[i] = titles_string[i].replace('smart hdtv', 'smartHdtv')
    titles_string[i] = titles_string[i].replace('smart led hdtv', 'smartLedHdtv')
    titles_string[i] = titles_string[i].replace('led hdtv', 'ledHdTv')
    titles_string[i] = titles_string[i].replace('hd tv', 'hdtv')
    titles_string[i] = titles_string[i].replace('ultra hdtv', 'ultraHdTv')

    titles_string[i] = titles_string[i].replace('smart 3d hdtv', 'smart3DHdtv')
    titles_string[i] = titles_string[i].replace('smart 3d plasma tv', 'smart3DPlasmatv')
    titles_string[i] = titles_string[i].replace(" hz", "hz") #transform to hz
    titles_string[i] = titles_string[i].replace("-", "")    


        
print(titles_string[1:4])



['newegg.com  supersonic 32inch 720p ledHdTv sc3211', 'sharp aquos 90inch class 90inch led 1080p 120hz smart3DHdtv lc90le657u  best buy', 'philips 2000 series 39inch class 3858inch led 1080p 60hz smartHdtv 39pfl2908f7  best buy']


# Use words in titles only

In [10]:
##Makes the titles of the products into a (by word callable list of strings)
def keyWordMaker(indexTitles):
    keyTitles = (titles_string[indexTitles])
    keyTitleSplit = keyTitles.split() 
    return keyTitleSplit

print(keyWordMaker(6))

['samsung', '40inch', 'class', '40inch', 'ledlcd', 'tv', '1080p', 'hdtv', '1080p', 'hg40na577lf', 'best', 'buy']


In [11]:
#add all individual lists of strings together in one list (callable by product index)
totalWords = []
for i in range(len(x_data)):
    totalWords.append(keyWordMaker(i))

#Remove all double keywords from totalWords
allKeyWords = []
for i in range(len(df)):
    for j in range(len(totalWords[i])):
        if totalWords[i][j] not in allKeyWords:
            allKeyWords.append(totalWords[i][j])
    if listIDs[i] in allKeyWords: ##remove modelID's that are in titles
        allKeyWords.remove(listIDs[i])        
    
allKeyWords.remove('class'); allKeyWords.remove('newegg.com')
allKeyWords.remove('best'); allKeyWords.remove('buy')

print("Length keywords using nonduplicates from solely the titles of x_data:", len(allKeyWords))
# allKeyWords

Length keywords using nonduplicates from solely the titles of x_data: 642


In [12]:
## Construct frequency of words to in-/exclude from keywords


# counts = Counter(totalWords[0])
# print(counts)
# Counter({'apple': 3, 'egg': 2, 'banana': 1})

# Create matrices for input matrix

In [13]:
#make single list containing all keywords ['', '',  , ..., ''] --> allKeyWordsSep
print(len(allKeyWords)) #-->p
# allKeyWordsSep
print(len(totalWords))

642
1624


In [14]:
##create inputMatrix with binary vectors for each keyword (rows) in each product (columns)
inputMatrix = np.zeros((len(allKeyWords), len(x_data)))


for k in range(len(allKeyWords)):
    for i in range(len(x_data)):
            if allKeyWords[k] in totalWords[i]:
                    inputMatrix[k][i] = 1

In [15]:
print('Columns of inputmatrix (should equal #products):', len(inputMatrix[0]))
print('Length of inputmatrix (=rows (should equal #keywords)):', len(inputMatrix))
inputMatrix

Columns of inputmatrix (should equal #products): 1624
Length of inputmatrix (=rows (should equal #keywords)): 642


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

## Minhasing


Random Numbers & Signature Matrix

In [16]:
#Random number generator function with integer as input
def randomNumberGenerator(value):
    #create list to store random values such that random values are all created once when function is called
    randomNumbers = []
    indexValue = random.randint(0, len(allKeyWords) - 1)
    randomNumbers.append(indexValue)
    
    while value > 0:
        while indexValue in randomNumbers:
            indexValue = random.randint(0, len(allKeyWords) - 1)
        randomNumbers.append(indexValue)
        value=value-1
    return randomNumbers

numMinHash = 500
p = 1543

In [17]:
#In separate cell call random number generator function

randomIntA = randomNumberGenerator(numMinHash)
randomIntB = randomNumberGenerator(numMinHash)

In [18]:
signatureMatrix = []

for i in range(len(inputMatrix[0])): #first loop over all products
    signature = []

    for j in range(0, numMinHash): #Do this 500 times
        minHashValue = 2**32 - 1
        
        for k in range(len(allKeyWords)): #then loop over all keywords
            if inputMatrix[k][i] == 1:
            
                hashValue = (randomIntA[j] + k * randomIntB[j]) % p

                if hashValue < minHashValue:                    
                    minHashValue = hashValue 
                    
                    
        signature.append(minHashValue)     
    signatureMatrix.append(signature)

In [19]:
signatureMatrixFin = np.transpose(pd.DataFrame(signatureMatrix))                    
print(len(signatureMatrixFin))
print(len(signatureMatrixFin[0])) #???

signatureMatrixFin

500
500


,0,1,2,3,4,5,6,7,8,9,...,1614,1615,1616,1617,1618,1619,1620,1621,1622,1623
0,95,261,51,95,217,300,300,300,7,300,...,256,256,300,300,300,510,300,300,104,383
1,196,153,110,67,24,324,24,24,24,238,...,453,453,409,409,238,109,410,410,410,24
2,52,446,52,52,63,260,85,85,85,446,...,249,249,52,52,446,249,155,155,85,85
3,34,188,265,111,14,91,91,91,168,91,...,516,516,91,91,91,265,91,91,188,265
4,45,143,241,298,45,86,45,45,45,86,...,45,45,45,45,143,45,143,143,143,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,54,252,54,54,95,491,177,177,177,20,...,252,252,20,20,20,491,450,450,177,177
496,14,110,46,14,78,270,78,78,78,110,...,78,78,46,46,110,78,110,110,110,78
497,11,88,165,281,165,49,49,589,126,358,...,88,88,162,162,358,87,49,49,88,126
498,119,303,34,119,310,402,402,41,133,402,...,303,303,140,140,402,55,402,402,175,494


## LSH

n = b * r, where n = length of columns of signature matrix M 

In [20]:
def threshold(value):
    t = (1/value[0]) ** (1/value[1])
    return t

In [21]:
# b = first value
# r = second value
thresholdPairs = [(2, 250), (4, 125), (5, 100), (10, 50), (20, 25), (100, 5)]
for i in thresholdPairs:
    print(i, threshold(i))
r = 100
b = 5
print('b = 5 and r = 100')

(2, 250) 0.9972312513520695
(4, 125) 0.9889709163809315
(5, 100) 0.9840344433634576
(10, 50) 0.954992586021436
(20, 25) 0.8870718549931568
(100, 5) 0.39810717055349726
b = 5 and r = 100


In [22]:
##Create bands with correct rows per band
z = np.arange(0,numMinHash,r)
bandsDict = {}
for i in range(len((z))-1):
#     value[j] = signatureMatrixFin[z[i]:z[i+1]][:]
    
    bandsDict["band{0}".format(i)]=signatureMatrixFin[z[i]:z[i+1]][:]
    
bandsDict["band{0}".format(b-1)]=signatureMatrixFin[z[b-1]:][:]    
    
# display(bandsDict['band0'])
display(bandsDict['band1'])
# display(bandsDict['band2'])
# display(bandsDict['band3'])



,0,1,2,3,4,5,6,7,8,9,...,1614,1615,1616,1617,1618,1619,1620,1621,1622,1623
100,96,245,96,96,112,112,128,692,277,245,...,11,11,6,6,245,6,128,128,6,277
101,141,263,324,202,45,106,106,106,167,106,...,332,332,106,106,106,280,106,106,263,324
102,84,810,265,84,83,265,83,83,83,446,...,447,447,354,444,718,170,84,84,265,83
103,146,217,19,146,19,19,430,430,232,217,...,79,79,146,146,217,176,217,217,19,430
104,194,186,178,170,178,162,178,154,154,146,...,186,186,178,178,146,98,178,178,178,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,15,206,108,10,108,108,108,103,108,5,...,206,206,5,5,5,98,108,108,108,108
196,201,506,189,201,183,177,177,171,482,476,...,39,39,500,500,476,153,177,177,500,159
197,15,15,138,323,76,261,76,14,76,199,...,15,15,322,627,199,507,447,447,129,76
198,245,261,277,245,277,285,285,285,285,285,...,261,261,285,285,285,285,285,285,285,285


In [23]:
##create hash values from bands
values = bandsDict.values()
keys = bandsDict.keys() #keys = ['band0', 'band1', ... , 'band124']
hashFuncDic = {}
((bandsDict['band0'][:][0])[0])

95

In [24]:
keys

dict_keys(['band0', 'band1', 'band2', 'band3', 'band4'])

In [25]:

##Works:

In [26]:
hashValBand = {}

for i in range(0,b):
    hashValBandS[i] = {}
    
    for j in range(len(inputMatrix[0])):
        valueHash1 = bandsDict['band'+str(i)][:][j]
        valueHash2 = [str(int) for int in valueHash1]
        valueHash3 = ''.join(valueHash2)
        if valueHash3 not in hashValBandS[i].keys():
            hashValBandS[i][valueHash3] = [j]
        else:
            hashValBandS[i][valueHash3].append(j)
            
            
    hashValBandDupl[i] = {}
    hashValBandNoDupl[i] = {}

    for key in hashValBandS[i]:
        if len(hashValBandS[i][key]) > 1:
              hashValBandDupl[i][key] = hashValBandS[i][key]
        else:
            hashValBandNoDupl[i][key] = hashValBandS[i][key]
        
##hashValBandDupl[0 to b] returns hashvalues&prodloc for products with
#equal hash values

        
# hashValBandNoDupl[1]


NameError: name 'hashValBandS' is not defined

In [ ]:
##Extract all candidate pairs of all bands for further comparison
candidateDict = {}
for i in range(0,b):
    
    candidateDict['candidatesBand%s' % i] = list(hashValBandDupl[i].values())
y_data[candidateDict['candidatesBand0'][0][1]] == y_data[candidateDict['candidatesBand0'][0][2]]

In [ ]:
## check whether the found candidate pairs are actually identical

# candidateDict['candidatesBand0'] == candidateDict['candidatesBand4']
# print(len(candidateDict['candidatesBand0']))
FP = 0
TP = 0
A = []
for i in range(len(candidateDict['candidatesBand0'])):
#     print([candidateDict['candidatesBand0'][i]])
#     print('')
    B = []
    for j in range(len(candidateDict['candidatesBand0'][i])):
        Q = [y_data[candidateDict['candidatesBand0'][i][j]]]
        B.append(Q)
    A.append(B)

A

In [ ]:
for i in range(len(A)):
    print(A[i])
    for j in A[i]:
        print(j)
        
#         if A[i]

In [ ]:
## for all candidate pairs of band0:
# print(candidateDict['candidatesBand0'])

##for first set of candidate pairs of band 0 --> [6, 18, 915]
print(candidateDict['candidatesBand1'][0])

#For first candidate of first candidate pairs of band 0 --> 6
print(candidateDict['candidatesBand0'][0][0])

y_data.iloc[candidateDict['candidatesBand0'][1]]
# y_train.iloc[[6, 18, 915]]



In [ ]:
giveJoinValues(0)


In [ ]:
giveJoinValues(1)[0]

    


In [ ]:
# hashValBandDupl
# hashValBandNoDupl

In [ ]:

def hashValAllBands():
    hashValBandDupl = {}
    hashValBandNoDupl = {}
    for key in hashValBand:
        if len(hashValBand[key]) > 1:
              hashValBandDupl[key] = hashValBand[key]
        else:
            hashValBandNoDupl[key] = hashValBand[key]

In [ ]:
giveJoinValues(0)

In [ ]:
for i in range(len(titles_string)):
    signature1 = sigComplete[i]
    
    for j in range(i+1, len(titles_string)):
        signature2 = sigComplete[j]
        
        count = 0
        for k in range(0, len(removeDupl)):
            count = count + (signature1[k] == signature2[k])

In [ ]:
for i in range(0, numDocs):
  # Get the MinHash signature for document i.
  signature1 = signatures[i]
    
  # For each of the other test documents...
  for j in range(i + 1, numDocs):
    
    # Get the MinHash signature for document j.
    signature2 = signatures[j]
    
    count = 0
    # Count the number of positions in the minhash signature which are equal.
    for k in range(0, numHashes):
        count = count + (signature1[k] == signature2[k])
    
    # Record the percentage of positions which matched.    
    estJSim[getTriangleIndex(i, j)] = (count / numHashes)

In [ ]:
# numMinHash = 100
# sigComplete = []
# for i in range(len(inputMatrix[0])): #titles --> columns
#     sig = []
#     for j in range(numMinHash):
#         minHashValue = 2**32 - 1
#         for k in range(len(removeDupl)): #keywords --> rows
#             if inputMatrix[k][i] == 1:
#                 hashValue = (randomNoA[k] + k*randomNoB[k]) % p
#                 if hashValue < minHashValue:
#                     minHashValue = hashValue
        
#         sig.append(minHashValue)
#     sigComplete.append(sig)
    